# FTP Transfer of NYSE, NASDAQ and Other Market Tickers

In [16]:
from ftplib import FTP
import pandas_datareader.data as web
from datetime import datetime, timedelta
import pandas as pd
import pickle
import yfinance as yf
import sqlite3 as sq3

In [2]:
directory = 'symboldirectory'
filenames = ('otherlisted.txt', 'nasdaqlisted.txt')

ftp = FTP('ftp.nasdaqtrader.com')
ftp.login()
ftp.cwd(directory)

# list the files on the FTP website
#ftp.retrlines('LIST')

for item in filenames:
    ftp.retrbinary(f'RETR {item}', 
    open(f'../../../data/{item}', 'wb').write)

ftp.quit()

# Create pandas dataframes from the nasdaqlisted and otherlisted files.
nasdaq_exchange_info = pd.read_csv('../../../data/nasdaqlisted.txt', '|')
other_exchange_info = pd.read_csv('../../../data/otherlisted.txt', '|')

C:\Users\Eddie\anaconda3\envs\securities-analysis\lib\site-packages\IPython\core\interactiveshell.py:3441: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
nasdaq_exchange_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5008 entries, 0 to 5007
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Symbol            5008 non-null   object 
 1   Security Name     5007 non-null   object 
 2   Market Category   5007 non-null   object 
 3   Test Issue        5007 non-null   object 
 4   Financial Status  5007 non-null   object 
 5   Round Lot Size    5007 non-null   float64
 6   ETF               4998 non-null   object 
 7   NextShares        5007 non-null   object 
dtypes: float64(1), object(7)
memory usage: 313.1+ KB


In [4]:
nasdaq_exchange_info.head()

,Symbol,Security Name,Market Category,Test Issue,Financial Status,Round Lot Size,ETF,NextShares
0,AACG,ATA Creativity Global - American Depositary Sh...,G,N,N,100.0,N,N
1,AACIU,Armada Acquisition Corp. I - Unit,G,N,N,100.0,N,N
2,AADI,"Aadi Bioscience, Inc. - Common Stock",S,N,N,100.0,N,N
3,AADR,AdvisorShares Dorsey Wright ADR ETF,G,N,N,100.0,Y,N
4,AAL,"American Airlines Group, Inc. - Common Stock",Q,N,N,100.0,N,N


In [5]:
# contains the tickers from NYSE MKT (Exchange = A),
# New York Stock Exchange (NYSE) (Exchange = N),
# NYSE ARCA (Exchange = P),
# BATS Global Markets (Exchange = BATS),
# Investors' Exchange (Exchange = IEXG)
other_exchange_info.head()

,ACT Symbol,Security Name,Exchange,CQS Symbol,ETF,Round Lot Size,Test Issue,NASDAQ Symbol
0,A,"Agilent Technologies, Inc. Common Stock",N,A,N,100.0,N,A
1,AA,Alcoa Corporation Common Stock,N,AA,N,100.0,N,AA
2,AAA,Listed Funds Trust AAF First Priority CLO Bond...,P,AAA,Y,100.0,N,AAA
3,AAAU,Goldman Sachs Physical Gold ETF Shares,P,AAAU,Y,100.0,N,AAAU
4,AAC,Ares Acquisition Corporation Class A Ordinary ...,N,AAC,N,100.0,N,AAC


## Download NASDAQ History

In [6]:
# download the whole daily history of the nasdaq
# SLOWS DOWN COMPUTER

# list of all the tickers in the nasdaq
nasdaq_tickers = list(nasdaq_exchange_info['Symbol'])
nasdaq_hist = yf.download(nasdaq_tickers)

nasdaq_hist = nasdaq_hist.stack(1).reset_index().rename(columns = {'level_1':'Ticker'}).set_index('Date')
nasdaq_hist.head()

# write to database (save to SSD)
con = sq3.connect('S://Data/securities_analysis_data/nasdaq_history.sql')
nasdaq_hist.to_sql('nasdaq_hist', con)

# SLOWER METHOD OF DOWNLOAD HISTORICAL DATA
# tickerStrings = nasdaq_tickers
# df_list = list()
# for ticker in tickerStrings:
#     data = yf.download(ticker, group_by="Ticker")
#     data['Ticker'] = ticker  # add this column because the dataframe doesn't contain a column with the ticker
#     df_list.append(data)
# nasdaq_hist = pd.concat(df_list)

[*********************100%***********************]  5008 of 5008 completed

449 Failed downloads:
- IPVIW: 1d data not available for startTime=-2208988800 and endTime=1632005342. Only 100 years worth of day granularity data are allowed to be fetched per request.
- CRTDW: 1d data not available for startTime=-2208988800 and endTime=1632005343. Only 100 years worth of day granularity data are allowed to be fetched per request.
- LATNW: 1d data not available for startTime=-2208988800 and endTime=1632005343. Only 100 years worth of day granularity data are allowed to be fetched per request.
- MSDAW: 1d data not available for startTime=-2208988800 and endTime=1632005344. Only 100 years worth of day granularity data are allowed to be fetched per request.
- CLOER: 1d data not available for startTime=-2208988800 and endTime=1632005345. Only 100 years worth of day granularity data are allowed to be fetched per request.
- IMTXW: 1d data not available for startTime=-2208988800 and endTime=16320053

NameError: name 'historical_data' is not defined

In [23]:
nasdaq_hist.head()

,Ticker,Adj Close,Close,High,Low,Open,Volume
Date,,,,,,,
1962-01-02,GT,1.936024,11.12500,11.18750,11.00000,0.0,32000.0
1962-01-03,GT,1.903394,10.93750,11.09375,10.93750,0.0,28000.0
1962-01-04,GT,1.892518,10.87500,11.06250,10.78125,0.0,29200.0
1962-01-05,GT,1.914269,11.00000,11.09375,10.87500,0.0,28000.0
1962-01-08,GT,1.876202,10.78125,11.00000,10.71875,0.0,43200.0


In [56]:
# query the nasdaq db

con = sq3.connect('S://Data/securities_analysis_data/nasdaq_history.sql')
query = '''
SELECT * FROM nasdaq_hist
WHERE Ticker = 'TSLA'
AND Volume > 1000000
LIMIT 10
'''

con = sq3.connect('S://Data/securities_analysis_data/nasdaq_history.sql')

test_tickers = ['TSLA']
end = datetime.now()
start = end - timedelta(days=365)

query = f'''
SELECT * FROM nasdaq_hist
WHERE Ticker = '{test_tickers[0]}'
AND DATE(Date) BETWEEN '{str(start.date())}'
AND '{str(end.date())}'
LIMIT 10
'''

query = '''
SELECT * FROM nasdaq_hist
WHERE Ticker = 'AACG'
LIMIT 10
'''

# query = f'''
# SELECT DATE(Date) FROM nasdaq_hist
# WHERE DATE(Date) = '2020-09-19'
# LIMIT 10
# '''

# query = f'''
# SELECT MAX(DATE(Date)) FROM nasdaq_hist
# '''

%time df = pd.read_sql_query(query, con)
df.tail()
df['Close'].iloc[-1]
# %time res = con.execute(query).fetchall()
# res

Wall time: 771 ms


9.039999961853027

In [14]:
con.close()

## Find all stock symbols that are beating the S&P 500 by more 
## than 10% over the last 365 days and store the last 365 days worth
## of data for those stock tickers into a python pickle.
## Use SPY for the ETF which tracks the S&P 500

In [39]:
# Total return of SP 500 over the last year
end = datetime.now()
start = end - timedelta(days=365)

valid_sources = ('morningstar', 'iex', 'yahoo')
close_string = 'Close'
source = valid_sources[2]

spy = web.DataReader('SPY', source, start, end)
spy_percent_change = (spy.iloc[-1][close_string] - spy.iloc[0][close_string]) / \
    spy.iloc[0][close_string] * 100

In [40]:
print(f"Essentially, S&P 500 has changed by {round(spy_percent_change,2)}% over the last year")

Essentially, S&P 500 has changed by 35.0% over the last year


In [57]:
# Loop through NASDAQ tickers and store stocks which have performed greater than the S&P 500 by at least 10%
# in the last year

# connect to equities database
con = sq3.connect('S://Data/securities_analysis_data/nasdaq_history.sql')

# test
#nasdaq_tickers = ['TSLA','AAPL']
end = datetime.now()
start = end - timedelta(days=365)

growth_stocks = dict()
for ticker in nasdaq_tickers:
    try:
        print(f'Checking stock: {ticker}')
        # download from yfinance
        #data = yf.download(ticker, start, end, progress=False)
        # query from the database
        query = f'''
        SELECT * FROM nasdaq_hist
        WHERE Ticker = '{ticker}'
        AND DATE(Date) BETWEEN '{str(start.date())}'
        AND '{str(end.date())}'
        '''
        data = pd.read_sql_query(query, con)
        last_close = data['Close'].iloc[-1]
        if last_close > 10.0: # stocks cheaper than this are sketch
            start_close = data['Close'][0]
            percent_change = (last_close - start_close) / start_close * 100
            #print(f"Percent change of {ticker} in the last year is: {round(percent_change,2)}%")
            #print(f"Difference between this annual total return and that of S&P 500 is: \
            #{round(percent_change - spy_percent_change, 2)}%")
            if percent_change - spy_percent_change > 10:
                growth_stocks[ticker] = data
                #print(growth_stocks.keys())
    except Exception as e:
        print(e)

pickle.dump(growth_stocks, open('S://Data/securities_analysis_data/nasdaq_growth_2021.p', 'wb'))

Checking stock: AACG
2.680000066757202
Checking stock: AACIU
9.920000076293945
Checking stock: AADI
29.350000381469727
Checking stock: AADR
65.93710327148438
Checking stock: AAL


KeyboardInterrupt: 